In [ ]:
import psycopg2
import pandas as pd
import json

config_file = "./modules/configs.json"
with open(config_file, 'r') as file:
    config = json.load(file)

connections = config.get("connections")
postgres = connections.get("postgres")
dbname = postgres.get("dbname")
user = postgres.get("user")
password = postgres.get("password")
host = postgres.get("host")
port = postgres.get("port")

# PostgreSQL connection details
conn = psycopg2.connect(
    dbname=dbname,
    user=user,
    password=password,
    host=host,
    port=port
)

In [ ]:
cursor = conn.cursor()

In [ ]:
# Function to load CSV data into PostgreSQL table
def load_csv_to_postgresql(table_name, csv_file):
    with open(csv_file, 'r') as f:
        cursor.copy_expert(f"COPY {table_name} FROM STDIN WITH CSV HEADER", f)
    conn.commit()

# Create tables
create_tables_sql = """
CREATE TABLE IF NOT EXISTS distributors (
    distributor_id UUID PRIMARY KEY,
    distributor_name VARCHAR(100),
    contact_name VARCHAR(50),
    contact_phone VARCHAR(50),
    address TEXT,
    country VARCHAR(100)
);

CREATE TABLE IF NOT EXISTS products (
    product_id UUID PRIMARY KEY,
    product_name VARCHAR(100),
    category VARCHAR(50),
    price NUMERIC(10, 2),
    distributor_id UUID,
    FOREIGN KEY (distributor_id) REFERENCES distributors(distributor_id)
);

CREATE TABLE IF NOT EXISTS inventory (
    inventory_id UUID PRIMARY KEY,
    distributor_id UUID,
    product_id UUID,
    quantity_available INTEGER,
    sale_date DATE,
    FOREIGN KEY (distributor_id) REFERENCES distributors(distributor_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id)
);

CREATE TABLE IF NOT EXISTS customers (
    customer_id UUID PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    email VARCHAR(100),
    phone_number VARCHAR(50),
    address TEXT,
    registration_date DATE,
    country VARCHAR(100)
);

CREATE TABLE IF NOT EXISTS orders (
    order_id UUID PRIMARY KEY,
    customer_id UUID,
    order_date DATE,
    order_status VARCHAR(20),
    total_amount NUMERIC(10, 2),
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
);

CREATE TABLE IF NOT EXISTS order_items (
    order_item_id UUID PRIMARY KEY,
    order_id UUID,
    product_id UUID,
    quantity INTEGER,
    price NUMERIC(10, 2),
    distributor_id UUID,
    purchase_date DATE,
    FOREIGN KEY (order_id) REFERENCES orders(order_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id),
    FOREIGN KEY (distributor_id) REFERENCES distributors(distributor_id)
);
"""

# Execute table creation
cursor.execute(create_tables_sql)
conn.commit()

# Load data into tables in the correct order
load_csv_to_postgresql('distributors', 'distributors.csv')
load_csv_to_postgresql('products', 'products.csv')
load_csv_to_postgresql('customers', 'customers.csv')
load_csv_to_postgresql('orders', 'orders.csv')
load_csv_to_postgresql('order_items', 'order_items.csv')
load_csv_to_postgresql('inventory', 'inventory.csv')


In [ ]:
try:
    # Drop tables in the correct order to avoid foreign key constraint violations
    drop_tables_sql = """
    DROP TABLE IF EXISTS order_items;
    DROP TABLE IF EXISTS orders;
    DROP TABLE IF EXISTS inventory;
    DROP TABLE IF EXISTS products;
    DROP TABLE IF EXISTS customers;
    DROP TABLE IF EXISTS distributors;
    """

    # Execute drop table commands
    cursor.execute(drop_tables_sql)
    conn.commit()

except Exception as e:
    print(f"Error occurred: {e}")
    conn.rollback()


In [ ]:
# Close the cursor and connection
cursor.close()
conn.close()